In [16]:
#!pip install autogluon 
#!pip install ccxt 
#!pip install pandas-ta

In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import numpy as np 
import pandas as pd 
import pandas_ta as ta
import ccxt
from datetime import datetime

In [2]:
chart_df = pd.read_feather('BTC_USDT-4h.feather')
chart_df['date'] = pd.to_datetime(chart_df['date'])
chart_df['date'] = chart_df['date'].dt.tz_localize(None) 

# define targets 
close = chart_df.close.values 
targets = [] 

for i in range(len(close) - 1): 
    targets.append(close[i+1]) 
targets.append(None) 
chart_df["targets"] = targets 
chart_df.dropna(inplace=True)

In [3]:
chart_df["item_id"] = "BTC-USDT"

In [4]:
chart_df

,date,open,high,low,close,volume,targets,item_id
0,2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865,4427.30,BTC-USDT
1,2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882,4352.34,BTC-USDT
2,2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001,4325.23,BTC-USDT
3,2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716,4285.08,BTC-USDT
4,2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913,4292.39,BTC-USDT
...,...,...,...,...,...,...,...,...
13713,2023-11-22 08:00:00,36439.80,36773.53,36391.03,36463.93,4677.776400,36481.14,BTC-USDT
13714,2023-11-22 12:00:00,36463.94,36650.00,36239.20,36481.14,6621.289880,37289.98,BTC-USDT
13715,2023-11-22 16:00:00,36481.15,37431.21,36372.58,37289.98,10097.378810,37408.34,BTC-USDT
13716,2023-11-22 20:00:00,37289.98,37861.10,37214.81,37408.34,10963.014440,37299.69,BTC-USDT


In [5]:
chart_df.dtypes

date       datetime64[ns]
open              float64
high              float64
low               float64
close             float64
volume            float64
targets           float64
item_id            object
dtype: object

In [6]:
train_data = TimeSeriesDataFrame.from_data_frame(
    chart_df, 
    id_column="item_id", 
    timestamp_column="date", 
)
train_data.head()

open     high      low    close      volume  \
item_id  timestamp                                                             
BTC-USDT 2017-08-17 04:00:00  4261.48  4349.99  4261.32  4349.99   82.088865   
         2017-08-17 08:00:00  4333.32  4485.39  4333.32  4427.30   63.619882   
         2017-08-17 12:00:00  4436.06  4485.39  4333.42  4352.34  174.562001   
         2017-08-17 16:00:00  4352.33  4354.84  4200.74  4325.23  225.109716   
         2017-08-17 20:00:00  4307.56  4369.69  4258.56  4285.08  249.769913   

                              targets  
item_id  timestamp                     
BTC-USDT 2017-08-17 04:00:00  4427.30  
         2017-08-17 08:00:00  4352.34  
         2017-08-17 12:00:00  4325.23  
         2017-08-17 16:00:00  4285.08  
         2017-08-17 20:00:00  4292.39

In [7]:
train_data = train_data.convert_frequency(freq="4h")

In [8]:
train_data.freq

'4H'

In [9]:
predictor = TimeSeriesPredictor(
    prediction_length=1, 
    path="autogluon_BTC_v2", 
    target="targets", 
    eval_metric="MAE" 
)

In [10]:
predictor.fit(
    train_data, 
    presets="best_quality", 
    time_limit=2400,
)

Beginning AutoGluon training... Time limit = 2400s
AutoGluon will save models to 'autogluon_BTC_v2'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #46~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Jun 7 15:06:04 UTC 20
CPU Count:          8
GPU Count:          1
Memory Avail:       40.75 GB / 44.08 GB (92.4%)
Disk Space Avail:   96428.55 GB / 459494.84 GB (21.0%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'targets',
 'time_limit': 2400,
 'verbosity': 2}

Inferred time series frequency: '4H'
train_data contains missing values represented by NaN. The

In [17]:
# inference
# load saved checkpoint 
predictor = TimeSeriesPredictor.load("autogluon_BTC_v2")
predictor 

Loading predictor from path autogluon_BTC_v2


In [18]:
# get inference data 
bitget = ccxt.bitget()
ohlcv = bitget.fetch_ohlcv("BTC/USDT:USDT", "4h")
chart_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

In [19]:
def preprocess(df):
    bitget = ccxt.bitget()
    dates = df["timestamp"].values
    utc_timestamps = []
    for timestamp in dates:
        # Convert to ISO 8601 format
        date_string = bitget.iso8601(int(timestamp))
        # Convert to datetime object without timezone conversion
        date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
        # Format to string if needed (optional)
        formatted_utc_time = date_object.strftime("%Y-%m-%d %H:%M:%S")
        utc_timestamps.append(formatted_utc_time)
    # Update DataFrame
    df["date"] = utc_timestamps
    df = df.drop(columns={"timestamp"})
    return df

In [20]:
chart_df = preprocess(chart_df)

In [21]:
chart_df

,open,high,low,close,volume,date
0,41822.3,41908.8,41387.0,41524.7,11206.916,2023-12-05 04:00:00
1,41524.7,41762.7,41420.0,41725.0,8300.591,2023-12-05 08:00:00
2,41725.0,42487.9,41693.0,42252.3,23501.413,2023-12-05 12:00:00
3,42252.3,44179.0,42115.8,43821.9,46774.065,2023-12-05 16:00:00
4,43821.9,44518.2,43427.5,44071.4,18375.522,2023-12-05 20:00:00
...,...,...,...,...,...,...
95,43688.1,43759.1,43309.9,43671.0,30322.093,2023-12-21 00:00:00
96,43671.0,43898.4,43469.0,43775.9,24829.901,2023-12-21 04:00:00
97,43775.9,44146.5,43600.0,43956.8,24980.397,2023-12-21 08:00:00
98,43956.8,44269.6,43375.9,43833.0,30167.370,2023-12-21 12:00:00


In [22]:
close = chart_df.close.values 
targets = [] 

for i in range(len(close) - 1): 
    targets.append(close[i+1]) 
targets.append(None) 
chart_df["targets"] = targets

chart_df["item_id"] = "BTC-USDT"

In [23]:
chart_df.dropna(inplace=True)

In [29]:
test_data = TimeSeriesDataFrame.from_data_frame(
    chart_df.iloc[:-2], 
    id_column="item_id", 
    timestamp_column="date", 
)
test_data.head()

open     high      low    close     volume  \
item_id  timestamp                                                            
BTC-USDT 2023-12-05 04:00:00  41822.3  41908.8  41387.0  41524.7  11206.916   
         2023-12-05 08:00:00  41524.7  41762.7  41420.0  41725.0   8300.591   
         2023-12-05 12:00:00  41725.0  42487.9  41693.0  42252.3  23501.413   
         2023-12-05 16:00:00  42252.3  44179.0  42115.8  43821.9  46774.065   
         2023-12-05 20:00:00  43821.9  44518.2  43427.5  44071.4  18375.522   

                              targets  
item_id  timestamp                     
BTC-USDT 2023-12-05 04:00:00  41725.0  
         2023-12-05 08:00:00  42252.3  
         2023-12-05 12:00:00  43821.9  
         2023-12-05 16:00:00  44071.4  
         2023-12-05 20:00:00  43746.9

In [30]:
test_data.tail()

open     high      low    close     volume  \
item_id  timestamp                                                            
BTC-USDT 2023-12-20 12:00:00  42877.4  44336.0  42771.2  44062.4  46197.112   
         2023-12-20 16:00:00  44062.4  44276.9  43506.6  43738.9  33838.398   
         2023-12-20 20:00:00  43738.9  43824.2  43237.0  43688.1  24797.292   
         2023-12-21 00:00:00  43688.1  43759.1  43309.9  43671.0  30322.093   
         2023-12-21 04:00:00  43671.0  43898.4  43469.0  43775.9  24829.901   

                              targets  
item_id  timestamp                     
BTC-USDT 2023-12-20 12:00:00  43738.9  
         2023-12-20 16:00:00  43688.1  
         2023-12-20 20:00:00  43671.0  
         2023-12-21 00:00:00  43775.9  
         2023-12-21 04:00:00  43956.8

In [31]:
autogluon_yhat = predictor.predict(test_data)

Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [32]:
autogluon_yhat

,,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
item_id,timestamp,,,,,,,,,,
BTC-USDT,2023-12-21 08:00:00,43820.495567,43419.678633,43557.270854,43656.484526,43741.258951,43820.495567,43899.732183,43984.506607,44083.72028,44221.312501


In [33]:
chart_df.iloc[-2:]

,open,high,low,close,volume,date,targets,item_id
97,43775.9,44146.5,43600.0,43956.8,24980.397,2023-12-21 08:00:00,43833.0,BTC-USDT
98,43956.8,44269.6,43375.9,43833.0,30167.370,2023-12-21 12:00:00,43624.9,BTC-USDT
